<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# NLCD downsampling and reprojection

_by Jeremy Freeman (CarbonPlan), August 2, 2020_

This notebook downsamples and reprojects NLCD 30m rasters stored in Cloud
Optimized GeoTIFF into 30m, 250m, and 4000m GeoTIFFs.

**Inputs:**

- COG outputs from `01_nlcd_to_cogs.ipynb`

**Outputs:**

- COG outputs after downsampling and reprojection

**Notes:**

- Source CRS and projection extent come from NLCD


In [ ]:
import os

In [ ]:
from carbonplan_data.utils import projections, setup

workdir, upload = setup("jeremy")

In [ ]:
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

dst_profile = cog_profiles.get("deflate")

In [ ]:
import rasterio
from numpy import asarray, argmax

In [ ]:
def get_file(region, year, resolution, c=None):
    if c is not None:
        target = f"processed/nlcd/{region}/{resolution}m/{year}_c{c}.tif"
    else:
        target = f"processed/nlcd/{region}/{resolution}m/{year}.tif"
    return {
        "source": workdir / f"nlcd/{region}/30m/{year}.tif",
        "target": target,
    }

In [ ]:
def loadrio(f):
    src = rasterio.open(f)
    return src.read(1)

### downsample using mode


In [ ]:
for resolution in [4000]:
    for region in ["ak", "conus"]:
        if region == "ak":
            years = [2011, 2016]
        if region == "conus":
            years = [2001, 2004, 2006, 2008, 2011, 2013, 2016]
        for year in years:
            f = get_file(region, year, resolution)
            crs, extent = projections("albers", region)
            if resolution == 30:
                resampling = "near"
            else:
                resampling = "mode"
            cmd = (
                "gdalwarp "
                "-t_srs '%s' "
                "-te %s "
                "-tr %s %s "
                "-r %s "
                "%s "
                "%s"
            ) % (
                crs,
                extent,
                resolution,
                resolution,
                resampling,
                f["source"],
                "./raster.tif",
            )
            os.system(cmd)
            cog_translate("./raster.tif", "./raster.tif", dst_profile)
            upload("./raster.tif", f["target"])
            os.remove("./raster.tif")

### downsample using thresholding


In [ ]:
categories = [
    11,
    12,
    21,
    22,
    23,
    24,
    31,
    41,
    42,
    43,
    51,
    52,
    71,
    72,
    73,
    74,
    81,
    82,
    90,
    95,
]

In [ ]:
for resolution in [4000]:
    for region in ["ak", "conus"]:
        if region == "ak":
            years = [2011, 2016]
        if region == "conus":
            years = [2001, 2004, 2006, 2008, 2011, 2013, 2016]
        for year in years:
            fbase = get_file(region, year, resolution)
            src = rasterio.open(fbase["source"])
            band = src.read(1)
            profile = src.profile

            for category in categories:
                print(f"region {region} year {year} cat {category}")
                f = get_file(region, year, resolution, category)
                crs, extent = projections("albers", region)
                if resolution == 30:
                    resampling = "near"
                else:
                    resampling = "average"

                out = (band == category).astype(rasterio.uint8)

                with rasterio.open("./thresholded.tif", "w", **profile) as dst:
                    dst.write(out, 1)

                cmd = (
                    "gdalwarp "
                    "-t_srs '%s' "
                    "-te %s "
                    "-tr %s %s "
                    "-r %s "
                    "-ot Float32 "
                    "%s "
                    "%s"
                ) % (
                    crs,
                    extent,
                    resolution,
                    resolution,
                    resampling,
                    "./thresholded.tif",
                    "./raster.tif",
                )

                os.system(cmd)
                cog_translate("./raster.tif", "./raster.tif", dst_profile)
                upload("./raster.tif", f["target"])
                os.remove("./thresholded.tif")
                os.remove("./raster.tif")

## recreate mode using max


In [ ]:
categories = [
    11,
    12,
    21,
    22,
    23,
    24,
    31,
    41,
    42,
    43,
    51,
    52,
    71,
    72,
    73,
    74,
    81,
    82,
    90,
    95,
]

In [ ]:
for resolution in [4000]:
    for region in ["ak"]:
        if region == "ak":
            years = [2011, 2016]
        if region == "conus":
            years = [2001, 2004, 2006, 2008, 2011, 2013, 2016]
        for year in years:
            bands = asarray(
                [
                    loadrio(
                        workdir / f"nlcd/{region}/{resolution}m/{year}_c{c}.tif"
                    )
                    for c in categories
                ]
            )
            out = argmax(bands, axis=0)
            out_reindexed = argmax(bands, axis=0).astype(rasterio.uint8)
            for i, c in enumerate(categories):
                out_reindexed[out == i] = c
            out_reindexed[bands.sum(axis=0) == 0] = 0

            src = rasterio.open(
                workdir / f"nlcd/{region}/{resolution}m/{year}_c11.tif"
            )
            profile = src.profile
            profile.update(dtype=rasterio.uint8)

            with rasterio.open("raster.tif", "w", **profile) as dst:
                dst.write(out_reindexed, 1)

            cog_translate("./raster.tif", f"./{year}_cmax.tif", dst_profile)

            f = get_file(region, year, resolution, "max")
            upload(f"./{year}_cmax.tif", f["target"])
            os.remove("./raster.tif")